In [ ]:
import pandas as pd
from pathlib import Path
from omegaconf import OmegaConf
import itertools
from copy import deepcopy
CONFIG_PATH = Path('config.yaml')

config = OmegaConf.load(CONFIG_PATH)

In [12]:
from utils import *
from collections import defaultdict

result_dict = defaultdict(list)
for dir_name in Path('output').iterdir():
    if not dir_name.is_dir():
        continue

    config_path = dir_name / 'config.yaml'
    assert config_path.exists(), f'{config_path} does not exist'
    config = load_conf(dir_name / 'config.yaml')
    
    if 'model_train' not in config:
        continue

    for hyperparameter in config.model_train:
        if hyperparameter == 'params':
            continue
        result_dict[hyperparameter].append(config.model_train[hyperparameter])
    for hyperparameter in config.model_train.params:
        if hyperparameter == 'early_stopping_rounds':
            continue
        result_dict[hyperparameter].append(config.model_train.params[hyperparameter])
    if 'run_artist_name' not in config.feature_engineering:
        result_dict['run_artist_name'].append(False)
    for feature in config.feature_engineering:
        if feature.startswith('run_'):
            result_dict[feature].append(config.feature_engineering[feature])

    auroc = load_json(dir_name / 'val_auroc.json')
    result_dict['val_auroc'].append(auroc['val_auroc'])

result_df = pd.DataFrame(result_dict)
result_df.to_csv('output/result_analysis.csv', index=False)
result_df.head()


,early_stopping_rounds,num_boost_round,bagging_fraction,bagging_freq,feature_fraction,learning_rate,max_bin,max_depth,min_data_in_leaf,num_leaves,run_artist_name,run_composer,run_genre_id,run_lyricist,run_register_duration,val_auroc
0,10,200,0.8,5,0.8,0.1,255,31,20,1024,False,False,True,False,False,0.801187
1,20,200,0.8,5,0.8,0.1,255,15,20,1024,False,True,False,True,False,0.779579
2,10,200,0.8,5,0.8,0.1,255,31,20,1024,False,True,True,False,True,0.802331
3,10,200,0.8,5,0.8,0.1,255,63,20,1024,False,False,True,True,True,0.814616
4,10,200,0.8,5,0.8,0.1,255,31,20,1024,False,False,False,True,True,0.801424
